# Redes neuronales hibridas  para clasificación multiple


In [ ]:
# Install the relevant packages.
#%pip install --upgrade pip
#%pip install torch torchvision torchaudio
#%pip install cudaq -> ya viene instalado en el braket de AWS

In [ ]:
# Check installed clasico
import sys
import numpy as np
import matplotlib
print(f"Python version: {sys.version}")
print(f"NumPy version: {np.__version__}")
print(f"Matplotlib version: {matplotlib.__version__}")

import torch, torchvision, torchaudio
print("torch:", torch.__version__)
print("vision:", torchvision.__version__)
print("audio:", torchaudio.__version__)

In [ ]:

import cudaq
print(f"CUDAQ version: {cudaq.__version__}")
print(f"Running on target: {cudaq.get_target().name}")

In [ ]:
import sys
import cudaq

print(f"Running on target {cudaq.get_target().name}")
qubit_count = 2


@cudaq.kernel
def kernel():
    qubits = cudaq.qvector(qubit_count)
    h(qubits[0])
    for i in range(1, qubit_count):
        x.ctrl(qubits[0], qubits[i])
    mz(qubits)


result = cudaq.sample(kernel)
print(result)  # Example: { 11:500 00:500 }

### Configuración de Hiperparámetros

Para facilitar la experimentación, todos los hiperparámetros importantes se definen en la siguiente celda. Puedes modificar estos valores para ver cómo afectan el rendimiento y el tiempo de entrenamiento del modelo.

In [ ]:
# --- Hiperparámetros del Modelo y Entrenamiento ---
# Versión ligera: 4 qubits + medición AMM (12 features) + PQC en escalera poco profunda
n_qubits = 4
pqc_layers = 2  # profundidad baja para reducir llamadas cuánticas
n_samples_prueba = 2000  # subconjunto para experimentar rápido; pon 60000 para todo MNIST reducido
batch_size = 16
epochs = 15
learning_rate = 8e-2  # LR moderado para estabilidad con el bloque cuántico


### Estructura del modelo AMM ligero
- Entrada: imagen MNIST reducida 4×4 (16 px) -> encoder clásico `Flatten -> Linear(16→32) -> ReLU -> Linear(32→4)`; ángulos $\alpha = \arcsin(\sigma(h)) \in \mathbb{R}^4$ con $\sigma$ = sigmoid.
- PQC: escalera de profundidad $L=2$ con $n_q=4$ qubits y compuertas ZZ (descompuestas en CX-RZ-CX). Estado final $U_{\text{PQC}}(\theta)\, U_{\text{enc}}(\alpha)\, |0\rangle^{\otimes n_q}$.
- Medición AMM: se miden $\langle \sigma_x \rangle, \langle \sigma_y \rangle, \langle \sigma_z \rangle$ en cada qubit; $m_{\text{AMM}} \in \mathbb{R}^{3 n_q}$ (12 features para 4 qubits).
- Clasificador: `LayerNorm -> Linear(12→32) -> ReLU -> Linear(32→10)`; `CrossEntropyLoss` aplica softmax interno.

### Flujo de datos y etiquetado
- Etiquetas $y \in \{0,\dots,9\}$ provienen directamente de MNIST (sin one-hot).
- Forward: $z = f_{\text{class}}(m_{\text{AMM}})$; pred = argmax$(z)$; $\mathcal{L} = \text{CE}(z, y)$. Misma división train/val/test que en el resto del notebook.

### Diferencias con `red_hibrida.ipynb`
- Observables: allí se mide un solo observable ($\sum Z$ -> 1 feature); aquí AMM en 4 qubits entrega 12 features, más señal para el clasificador.
- Encoder: allí CNN completa 28×28 + FC grande; aquí reducción 4×4 + encoder pequeño, bajando cómputo y llamadas cuánticas.
- Gradientes: allí se propagan gradientes también a las entradas del PQC; aquí solo a $\theta$ del PQC (parameter-shift), lo que reduce ejecuciones cuánticas en backward.

### Optimización y parameter-shift
- Pérdida en un minibatch $B$: $\mathcal{L} = -\frac{1}{|B|} \sum_{i \in B} \log(\text{softmax}(z_i)_{y_i})$.
- Gradientes cuánticos via parameter-shift para cada $\theta_j$: $\partial_{\theta_j}\mathcal{L} \approx \tfrac{1}{2} [\mathcal{L}(\theta_j + \tfrac{\pi}{2}) - \mathcal{L}(\theta_j - \tfrac{\pi}{2})]$, aplicados solo a las $\theta$ del PQC; encoder y clasificador usan backprop estándar.
- Actualización con Adam (lr = $8\times10^{-4}$) y scheduler StepLR ($\gamma=0.2$ cada 7 épocas).


In [ ]:
# importar las librerías necesarias
import cudaq
from cudaq import spin

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

# Para asegurar que los resultados sean reproducibles
torch.manual_seed(33)

cudaq.set_random_seed(33)



In [ ]:
# Configurar el dispositivo
# Set CUDAQ and PyTorch to run on either CPU or GPU.
device = "cuda" if torch.cuda.is_available() else "cpu"
if device == "cuda":
    cudaq.set_target("nvidia")
    print("✅ Dispositivo configurado para GPU (CUDA).")
else:
    cudaq.set_target("qpp-cpu")
    print("⚠️ GPU no encontrada. Usando CPU.")


## Descripción del Conjunto de Datos MNIST

- *¿Qué es?:* MNIST (Modified National Institute of Standards and Technology database) es una gran base de datos de dígitos escritos a mano, del 0 al 9.
- *Contenido:* Contiene 70,000 imágenes en escala de grises.
- *Conjunto de entrenamiento:* 60,000 imágenes.
- *Conjunto de prueba:* 10,000 imágenes.
- *Formato de imagen:* Cada imagen tiene un tamaño de 28x28 píxeles.
- *Uso común:* Es considerado el "Hola, Mundo" de la visión por computadora y el aprendizaje profundo. Se utiliza para entrenar y probar algoritmos de clasificación de imágenes.


Paso 2: Cargar, Transformar y Previsualizar los Datos
torchvision nos facilita la descarga y preparación de datasets.

Transformaciones: Convertimos las imágenes a tensores de PyTorch y las normalizamos. La normalización (ajustar los valores de los píxeles para que tengan una media de 0.5 y una desviación estándar de 0.5) ayuda a que el modelo entrene más rápido y de forma más estable.
Descarga: Descargamos los conjuntos de entrenamiento y prueba. FashionMNIST ya viene separado en train y test.

In [ ]:
# Transformación 4x4 inspirada en la reducción del paper (Figura 2)
# 1) Normaliza a [0,1] con ToTensor
# 2) Recorta 4 píxeles por lado (28 -> 20)
# 3) Promedia cada bloque 5x5 para obtener una imagen 4x4
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Lambda(lambda t: t[:, 4:-4, 4:-4]),
    transforms.Lambda(lambda t: F.avg_pool2d(t, kernel_size=5, stride=5)),
])

# Descargar los datasets de entrenamiento y prueba con la transformación reducida
train_full_dataset = torchvision.datasets.MNIST(
    root='./data', 
    train=True, 
    download=True, 
    transform=transform
)

test_dataset = torchvision.datasets.MNIST(
    root='./data', 
    train=False, 
    download=True, 
    transform=transform
)

print(f"Tamaño total del dataset de entrenamiento: {len(train_full_dataset)}")
print(f"Tamaño del dataset de prueba: {len(test_dataset)}")
print(f"Dimensión tras reducción: {train_full_dataset[0][0].shape}")



In [ ]:
# Subconjunto configurable (por defecto se usa todo el dataset de entrenamiento reducido)
from torch.utils.data import Subset

train_subset_for_testing = Subset(train_full_dataset, range(n_samples_prueba))

train_size = int(0.8 * len(train_subset_for_testing))
val_size = len(train_subset_for_testing) - train_size

train_dataset, val_dataset = random_split(train_subset_for_testing, [train_size, val_size])

print(f"Tamaño del subconjunto de entrenamiento: {len(train_dataset)}")
print(f"Tamaño del subconjunto de validación: {len(val_dataset)}")

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)



In [ ]:
# Clases de MNIST (dígitos del 0 al 9)
classes = tuple(str(i) for i in range(10))

# Función para mostrar imágenes reducidas

def imshow(img):
    npimg = img.numpy()
    plt.imshow(np.squeeze(np.transpose(npimg, (1, 2, 0))), cmap='gray')
    plt.axis('off')
    plt.show()

# Obtener un lote de imágenes de entrenamiento
dataiter = iter(train_loader)
images, labels = next(dataiter)

# Mostrar las primeras 8 imágenes reducidas
imshow(torchvision.utils.make_grid(images[:8], nrow=8))
print('Etiquetas: ', ' '.join(f'{classes[labels[j]]}' for j in range(8)))



### Nota sobre hiperparámetros
Los hiperparámetros (incluido `n_samples_prueba`) se definen una sola vez en la celda superior. Ajusta ahí el valor (ej. 2000) y ejecuta en orden para que el subconjunto use ese número; no hay redefinición posterior.


In [ ]:
# --- Hiperparámetros del Modelo y Entrenamiento ---
# Versión ligera: 4 qubits + medición AMM (12 features) + PQC en escalera poco profunda
n_qubits = 4
pqc_layers = 2  # profundidad baja para reducir llamadas cuánticas
n_samples_prueba = 12000  # subconjunto para experimentar rápido; pon 60000 para todo MNIST reducido
batch_size = 64
epochs = 15
learning_rate = 8e-4  # LR moderado para estabilidad con el bloque cuántico


## Implementación del Modelo Híbrido Cuántico-Clásico

Ahora, construiremos y entrenaremos el modelo híbrido. Este modelo combinará capas convolucionales clásicas para la extracción de características con un circuito cuántico parametrizado para el procesamiento de la información.

In [ ]:
from torch.autograd import Function
from cudaq import spin

# Circuito AMM con pocos qubits y medición X/Y/Z en todos los qubits
ladder_depth = pqc_layers
theta_kernel, data_params, theta_params = cudaq.make_kernel(list, list)
qubits = theta_kernel.qalloc(n_qubits)

# Codificación por ángulos RY
for i in range(n_qubits):
    theta_kernel.ry(data_params[i], qubits[i])

# PQC en escalera con compuertas ZZ descompuestas en CX-RZ-CX
theta_index = 0
for _ in range(ladder_depth):
    for j in range(n_qubits - 1):
        theta_kernel.cx(qubits[j], qubits[j + 1])
        theta_kernel.rz(2 * theta_params[theta_index], qubits[j + 1])
        theta_kernel.cx(qubits[j], qubits[j + 1])
        theta_index += 1

# Medición X/Y/Z en todos los qubits -> 3 * n_qubits salidas
measurement_ops = []
for op in (spin.x, spin.y, spin.z):
    for q in range(n_qubits):
        measurement_ops.append(op(q))


def evaluate_expectations(sample_angles, theta_values):
    """Ejecuta el kernel y devuelve <X|Y|Z> de cada qubit."""
    obs_results = []
    for obs in measurement_ops:
        res = cudaq.observe(theta_kernel, obs, sample_angles, theta_values)
        obs_results.append(res.expectation())
    return obs_results


def evaluate_batch(data_tensor, theta_values, device):
    """Evalúa un batch completo para usar en forward y backward."""
    batch_results = []
    for row in data_tensor:
        batch_results.append(evaluate_expectations(row.detach().cpu().tolist(), theta_values))
    return torch.tensor(batch_results, device=device, dtype=data_tensor.dtype)


class QuantumFunction(Function):
    """Autograd manual: gradiente solo sobre parámetros cuánticos para ahorrar cómputo."""

    @staticmethod
    def forward(ctx, data: torch.Tensor, thetas: torch.Tensor):
        ctx.save_for_backward(data, thetas)
        theta_list = thetas.detach().cpu().tolist()
        return evaluate_batch(data, theta_list, data.device)

    @staticmethod
    def backward(ctx, grad_output: torch.Tensor):
        data, thetas = ctx.saved_tensors
        theta_list = thetas.detach().cpu().tolist()
        shift = np.pi / 2.0

        gradients_theta = torch.zeros_like(thetas)
        # Solo calculamos gradientes respecto a theta (la entrada no requiere gradiente)
        for t_idx in range(len(theta_list)):
            theta_plus = theta_list.copy()
            theta_minus = theta_list.copy()
            theta_plus[t_idx] += shift
            theta_minus[t_idx] -= shift

            exp_plus = evaluate_batch(data, theta_plus, data.device)
            exp_minus = evaluate_batch(data, theta_minus, data.device)
            gradient_component = 0.5 * (exp_plus - exp_minus)
            gradients_theta[t_idx] = (gradient_component * grad_output).sum()

        return None, gradients_theta


class QuantumLayer(nn.Module):
    """Capa cuántica con parámetros entrenables del PQC (profundidad en escalera)."""
    def __init__(self, n_qubits: int, ladder_depth: int):
        super().__init__()
        self.n_qubits = n_qubits
        self.ladder_depth = ladder_depth
        self.theta = nn.Parameter(torch.zeros((n_qubits - 1) * ladder_depth))

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return QuantumFunction.apply(x, self.theta)


class HybridAMM(nn.Module):
    """Modelo híbrido ligero: encoder clásico pequeño + PQC AMM + clasificador."""
    def __init__(self, n_qubits: int = n_qubits, ladder_depth: int = pqc_layers):
        super().__init__()
        self.n_qubits = n_qubits

        # Encoder clásico compacto para comprimir 4x4=16 features a 4 ángulos
        self.encoder = nn.Sequential(
            nn.Flatten(),
            nn.Linear(16, 32),
            nn.ReLU(),
            nn.Linear(32, n_qubits),
        )

        self.quantum = QuantumLayer(n_qubits, ladder_depth)

        # 3 observables por qubit -> 12 features para 4 qubits
        self.classifier = nn.Sequential(
            nn.LayerNorm(3 * n_qubits),
            nn.Linear(3 * n_qubits, 32),
            nn.ReLU(),
            nn.Linear(32, 10),
        )

    def forward(self, x):
        # Normaliza a [0,1] y usa arcsin como en el paper
        angles = torch.sigmoid(self.encoder(x))
        angles = torch.arcsin(torch.clamp(angles, 0.0, 1.0))

        q_out = self.quantum(angles)
        logits = self.classifier(q_out)
        return logits


In [ ]:
# --- Creación y Validación del Modelo ---

# Crear instancia del modelo híbrido ligero con AMM
hybrid_model = HybridAMM(n_qubits=n_qubits, ladder_depth=pqc_layers).to(device)
print("--- Arquitectura del Modelo Híbrido (AMM ligero) ---")
print(hybrid_model)

# --- Test de Validación Cuántico-Clásico (Forward y Backward) ---
print("--- Realizando Test de Validación Cuántico-Clásico ---")

test_input = torch.randn(4, 1, 4, 4).to(device)  # imágenes ya reducidas a 4x4
print(f"Input shape: {test_input.shape}")

output = hybrid_model(test_input)
print(f"Output shape: {output.shape}")
print("✅ Pase hacia adelante completado")

# Pase hacia atrás
target = torch.randint(0, 10, (4,)).to(device)
loss_fn_test = nn.CrossEntropyLoss()
loss = loss_fn_test(output, target)
loss.backward()

grad_pre_quantum = list(hybrid_model.quantum.parameters())[0].grad
if grad_pre_quantum is not None and grad_pre_quantum.abs().sum() > 0:
    print("✅ Backward funcionando: gradientes fluyen hasta el PQC y el clasificador.")
else:
    print("❌ Error en el backward: sin gradientes en el PQC.")


### Entrenamiento del Modelo Híbrido

Ahora, entrenaremos el modelo híbrido usando el mismo bucle de entrenamiento que para el modelo clásico. Notarás que el entrenamiento puede ser más lento debido a la simulación de los circuitos cuánticos.

In [ ]:
# --- Definir los bucles de entrenamiento y validación ---

def train_loop(dataloader, model, loss_fn, optimizer):
    """Bucle para una época de entrenamiento."""
    size = len(dataloader.dataset)
    model.train() # Poner el modelo en modo de entrenamiento
    total_loss = 0
    for batch, (X, y) in enumerate(dataloader):
        # Mover datos al dispositivo (CPU o GPU)
        X, y = X.to(device), y.to(device)

        # Calcular la predicción y la pérdida
        pred = model(X)
        loss = loss_fn(pred, y)
        total_loss += loss.item()

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 10 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
    return total_loss / len(dataloader)

def validation_loop(dataloader, model, loss_fn):
    """Bucle para evaluar el modelo en el conjunto de validación o prueba."""
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval() # Poner el modelo en modo de evaluación
    test_loss, correct = 0, 0

    with torch.no_grad(): # No necesitamos calcular gradientes aquí
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    return test_loss, correct

In [ ]:
# --- Entrenamiento del Modelo Híbrido ---
import time
# Definir la función de pérdida y el optimizador
loss_fn_hybrid = nn.CrossEntropyLoss()
optimizer_hybrid = optim.Adam(hybrid_model.parameters(), lr=learning_rate)

# Reducimos el learning rate en un factor de 0.2 cada 7 épocas
scheduler = torch.optim.lr_scheduler.StepLR(optimizer_hybrid, step_size=7, gamma=0.2)
# Listas para almacenar el historial de entrenamiento
history = {'train_loss': [], 'val_loss': [], 'val_accuracy': []}

# Bucle principal de entrenamiento para el modelo híbrido
print("
--- Iniciando Entrenamiento del Modelo Híbrido AMM con MNIST ---")
start_time = time.time()
for t in range(epochs):
    print(f"Época {t+1}
-------------------------------")
    print("Entrenando (Híbrido AMM)...")
    train_loss = train_loop(train_loader, hybrid_model, loss_fn_hybrid, optimizer_hybrid)
    print("Validando (Híbrido AMM)...")
    val_loss, val_acc = validation_loop(val_loader, hybrid_model, loss_fn_hybrid)
    
    # Guardar métricas de la época
    history['train_loss'].append(train_loss)
    history['val_loss'].append(val_loss)
    history['val_accuracy'].append(val_acc)
    scheduler.step()

end_time = time.time()
total_training_time = end_time - start_time
print("¡Entrenamiento híbrido finalizado!")

# Evaluar el modelo híbrido en el conjunto de prueba
print("
--- Evaluando Modelo Híbrido AMM con el conjunto de Prueba (Test) ---")
validation_loop(test_loader, hybrid_model, loss_fn_hybrid)

# Guardar los pesos del modelo híbrido entrenado
save_model(hybrid_model, "hybrid_amm_mnist_weights_gpu" if device=="cuda" else "hybrid_amm_mnist_weights_cpu")


In [ ]:
# --- Visualización de Resultados ---

import matplotlib.pyplot as plt

# Usamos las variables definidas en las celdas anteriores
num_epochs = epochs
num_train_samples = len(train_dataset)

# Determinar el nombre del dispositivo para el título de la gráfica
if device == "cuda":
    device_name = f"GPU ({torch.cuda.get_device_name(0)})"
else:
    device_name = "CPU"

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10), sharex=True)

# Título principal de la figura con la nueva información del dispositivo
title_text = "
".join([
    "Modelo Híbrido AMM - Historial de Entrenamiento",
    f"({num_epochs} épocas en {num_train_samples} muestras) batch_size={batch_size}",
    f"Ejecutado en: {device_name}",
    f"Tiempo total de entrenamiento: {total_training_time:.2f} segundos",
])
fig.suptitle(title_text, fontsize=16)

# Gráfica de Pérdida (Loss)
ax1.plot(history['train_loss'], label='Pérdida de Entrenamiento', marker='o')
ax1.plot(history['val_loss'], label='Pérdida de Validación', marker='o')
ax1.set_ylabel('Pérdida (Loss)')
ax1.set_title('Pérdida a lo largo de las Épocas')
ax1.legend()
ax1.grid(True)

# Gráfica de Precisión (Accuracy)
ax2.plot(history['val_accuracy'], label='Precisión de Validación', color='green', marker='o')
ax2.set_xlabel('Épocas')
ax2.set_ylabel('Precisión (Accuracy)')
ax2.set_title('Precisión de Validación a lo largo de las Épocas')
ax2.legend()
ax2.grid(True)

plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()


In [ ]:
import torch
import matplotlib.pyplot as plt
import random

hybrid_model.eval()
dataiter = iter(test_loader)
images, labels = next(dataiter)
images, labels = images.to(device), labels.to(device)

idx = random.randint(0, len(images) - 1)
img = images[idx].unsqueeze(0)
true_label = labels[idx]

with torch.no_grad():
    output = hybrid_model(img)
    _, predicted = torch.max(output, 1)

img_display = img.cpu().squeeze()
plt.imshow(img_display, cmap='gray')
plt.title(f"Etiqueta Real: {classes[true_label]} | Predicción: {classes[predicted.item()]}")
plt.axis('off')
plt.show()

